##  Tải các gói thư viện:

In [14]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install langchain
!pip install langchainhub
!pip install langchain-chroma
!pip install langchain-community
!pip install langchain-openai
!pip install langchain_hggingface
!pip install chainlit
!pip install python-dotenv
!pip install pypdf
!npm install localtunnel
!pip install numpy

ERROR: Could not find a version that satisfies the requirement langchain_hggingface (from versions: none)
ERROR: No matching distribution found for langchain_hggingface
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.439s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


## Import các gói thư viện cần thiết:

In [18]:
import chainlit as cl
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM ,pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

## Cài đặt lại các hàm và instance ở file trước: 

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
embedding = HuggingFaceEmbeddings()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Xây dựng hàm xử lý file input đầu vào

In [17]:
def process_file(file: AskFileResponse):
    if file.type == "text/plain":
        Loader = TextLoader
    elif file.type == "application/pdf":
        Loader = PyPDFLoader
    loader = Loader(file.path)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)
    for i, doc in enumerate(docs):
        doc.metadata["source"] = f"source_{i}"
    return docs

## Xây dựng hàm khởi tạo Chroma database:

In [19]:
def get_vector_db(file: AskFileResponse):
    docs = process_file(file)
    cl.user_session.set("docs",docs)
    vector_db = Chroma.from_documents(docs, embedding)
    return vector_db

## Khởi tạo mô hình ngôn ngữ lớn:

In [20]:
def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5", max_new_tokens: int = 512):
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=nf4_config,
        low_cpu_mem_usage=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens = max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        device_map="auto"
    )

    llm = HuggingFacePipeline(pipeline=model_pipeline)

    return llm
LLM = get_huggingface_llm()

## Khởi tạo welcome message:

In [23]:
welcome_message = """ Welcome to the PDF QA! To get started :
                  1. Upload a PDF or text file
                  2. Ask a question about the file
                  """

## Khởi tạo hàm on_chat_start

In [24]:
@cl.on_chat_start
async def on_chat_start():
    file = None
    while files is None:
        files = await cl.AskFileMessage(
            content = welcome_message,
            accept = ["text/plain", "application/pdf"],
            max_size_mb = 20,
            timeout = 180,
        ).send()
    file = files[0]

    msg = cl.Message(content=f"Processing '{file.name}' ... ", disable_feedback=True)
    await msg.send()

    vector_db = await cl.make_async(get_vector_db)(file)

    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        return_messages=True,
        chat_message_history=message_history,
    )
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 3})

    chain = ConversationalRetrievalChain.from_llm(
        llm=LLM,
        chain_type="stuff",
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
    )

    msg.content = f"'{file.name}' processed. You can now ask question !"
    await msg.update()

    cl.user_session.set("chain", chain)

## Khởi tạo hàm on_message

In [25]:
@cl.on_message
async def on_message (message : cl.Message):
    chain = cl.user_session.get (" chain ")
    cb = cl.AsyncLangchainCallbackHandler ()
    res = await chain.ainvoke(message.content, callbacks=[cb])
    answer = res["answer"]
    source_documents = res ["source_documents"]
    text_elements = []
    if source_documents :
        for source_idx, source_doc in enumerate (source_documents):
            source_name = f"source_{source_idx}"
            text_elements.append(
                cl.Text(content = source_doc.page_content, name = source_name)
            )
        source_names = [text_el.name for text_el in text_elements]
        if source_names:
            answer += f"\nSources: {', '.join(source_names)}"
        else:
            answer += "\nNo sources found"
    await cl.Message(content = answer, elements = text_elements).send ()

## Viết file app.py

In [26]:
%%writefile app.py

Writing app.py


## Cài đặt pyngrok

In [28]:
!pip install pyngrok -q

In [29]:
from pyngrok import ngrok

## Deploy

In [30]:
!ngrok config add-authtoken <<ngrok_id>>
public_url = ngrok.connect(8000).public_url
print(public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
https://5259-34-87-18-149.ngrok-free.app


In [ ]:
!chainlit run app.py

2024-07-06 19:00:02.310951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-06 19:00:02.310994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-06 19:00:02.312336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-06 19:00:03.442580: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-06 19:00:07 - Use pytorch device_name: cuda
2024-07-06 19:00:07 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarni